In [1]:
# Importação dos pacotes
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import csv
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

from __future__ import print_function
import time

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

#!pip install efficient-apriori
from efficient_apriori import apriori

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.dates as mdates

import seaborn as sns

from ydata_profiling import ProfileReport

# Seed para reprodução de resultados
seed = 1
random.seed(seed)
np.random.seed(seed)

In [2]:
#list all csv files only
csv_files = glob.glob('*.{}'.format('csv'))
csv_files

['baker1a_0-299_VS.csv',
 'baker1b_0-299_VS.csv',
 'baker2a_1200-1499_VS.csv',
 'baker2b_1200-1499_VS.csv',
 'baker3a_2700-2999_VS.csv',
 'baker3b_2700-2999_VS.csv',
 'baker4a_5100-5399_VS.csv',
 'baker4b_5100-5399_VS.csv',
 'baker5a_6000-6299_VS.csv',
 'baker5b_6000-6299_VS.csv',
 'baker6a_9000-9299_VS.csv',
 'baker6b_9000-9299_VS.csv',
 'baker7a_9300-9599_VS.csv',
 'baker7b_9300-9599_VS.csv']

In [3]:
df_append = pd.DataFrame()
#append all files together
for file in csv_files:
            df_temp = pd.read_csv(file,na_values=[-99999.25])
            df_append = df_append.append(df_temp, ignore_index=True)
df_append

C:\Users\gustavo.araujo\AppData\Local\Temp\ipykernel_14480\1762955736.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
C:\Users\gustavo.araujo\AppData\Local\Temp\ipykernel_14480\1762955736.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
C:\Users\gustavo.araujo\AppData\Local\Temp\ipykernel_14480\1762955736.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
C:\Users\gustavo.araujo\AppData\Local\Temp\ipykernel_14480\1762955736.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

,TIME (Date and Time),0 (Pod Select),16 (Pod 1 Flow Meter - Total 1),17 (Pod 2 Flow Meter - Total 1),45 (Alarm Lamp),47 (Alarm Horn),250 (Test Pod SEM-A Serial Software Process - CCU Node # - Primary),251 (Test Pod SEM-A Serial Software Process - CCU Node # - Secondary),252 (Test Pod SEM-B Serial Software Process - CCU Node # - Primary),253 (Test Pod SEM-B Serial Software Process - CCU Node # - Secondary),...,9546 (SFS Ext. Function: Pilot Supply),9547 (SFS Ext. Function: Manifold Regulated Supply),9550 (External Pod Command State - Solenoid 1),9551 (External Pod Command State - Solenoid 2),9552 (External Pod Command State - Solenoid 3),9553 (External Pod Command State - Solenoid 4),9554 (External Pod Command State - Solenoid 5),9555 (External Pod Command State - Solenoid 6),9556 (External Pod Command State - Solenoid 7),9557 (External Pod Command State - Solenoid 8)
0,2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-12-08T18:00:03.000Z,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-12-08T18:00:06.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-08T18:00:09.000Z,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-12-08T18:00:39.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127995,2022-12-10T00:36:03.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127996,2022-12-10T00:36:17.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127997,2022-12-10T00:36:25.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127998,2022-12-10T00:36:45.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Set date as Index
df_append = df_append.set_index('TIME (Date and Time)').drop_duplicates()
df_append = df_append.sort_index(axis=0)
df_append

,0 (Pod Select),16 (Pod 1 Flow Meter - Total 1),17 (Pod 2 Flow Meter - Total 1),45 (Alarm Lamp),47 (Alarm Horn),250 (Test Pod SEM-A Serial Software Process - CCU Node # - Primary),251 (Test Pod SEM-A Serial Software Process - CCU Node # - Secondary),252 (Test Pod SEM-B Serial Software Process - CCU Node # - Primary),253 (Test Pod SEM-B Serial Software Process - CCU Node # - Secondary),1 (Blue SEM-A;B Active/Standby and Online/Offline State),...,9546 (SFS Ext. Function: Pilot Supply),9547 (SFS Ext. Function: Manifold Regulated Supply),9550 (External Pod Command State - Solenoid 1),9551 (External Pod Command State - Solenoid 2),9552 (External Pod Command State - Solenoid 3),9553 (External Pod Command State - Solenoid 4),9554 (External Pod Command State - Solenoid 5),9555 (External Pod Command State - Solenoid 6),9556 (External Pod Command State - Solenoid 7),9557 (External Pod Command State - Solenoid 8)
TIME (Date and Time),,,,,,,,,,,,,,,,,,,,,
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:01.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:02.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-10T01:57:22.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-10T01:59:17.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-10T02:00:39.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Verificar atributos com muitos missing
percent_missing = df_append.apply(lambda x: x.isnull().sum() * 100 / len(df_append), axis=0).sort_values(ascending=False)
df = pd.DataFrame({'column_name': df_append.columns, 'percent_missing': percent_missing})
df_missing = pd.DataFrame({'column_name': df.index,'percent_missing': percent_missing})
df_no_data = df_missing[percent_missing > 99.999]
df_filtered = df_append.drop(columns= df_no_data['column_name'])
df_filtered

,0 (Pod Select),16 (Pod 1 Flow Meter - Total 1),17 (Pod 2 Flow Meter - Total 1),45 (Alarm Lamp),47 (Alarm Horn),250 (Test Pod SEM-A Serial Software Process - CCU Node # - Primary),251 (Test Pod SEM-A Serial Software Process - CCU Node # - Secondary),252 (Test Pod SEM-B Serial Software Process - CCU Node # - Primary),253 (Test Pod SEM-B Serial Software Process - CCU Node # - Secondary),1305 (SC Manifold Regulator Pressure - Decrease),...,9532 (SmV Ext. Function: Lower Inner Kill Valve - Close),9533 (SmV Ext. Function: Lower Inner Kill Valve - Open),9540 (SFS Ext. Function: Lower Outer Kill Valve - Close),9541 (SFS Ext. Function: Lower Outer Kill Valve - Open),9542 (SFS Ext. Function: Lower Inner Kill Valve - Close),9543 (SFS Ext. Function: Lower Inner Kill Valve - Open),9550 (External Pod Command State - Solenoid 1),9551 (External Pod Command State - Solenoid 2),9552 (External Pod Command State - Solenoid 3),9553 (External Pod Command State - Solenoid 4)
TIME (Date and Time),,,,,,,,,,,,,,,,,,,,,
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:01.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-08T18:00:02.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-10T01:57:22.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-10T01:59:17.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-10T02:00:39.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Verificar quais atributos mais frequentes da base
df_missing.tail(20)

,column_name,percent_missing
6209 (Diverter - Manifold Regulated Pressure (PT4-1)),6209 (Diverter - Manifold Regulated Pressure (...,96.697810
1409 (Upper Annular Regulator Readback Pressure),1409 (Upper Annular Regulator Readback Pressure),96.597960
9274 (Stack HPHT Pressure - Corrected (PSI)),9274 (Stack HPHT Pressure - Corrected (PSI)),96.597960
1406 (Manifold Regulator Readback Pressure),1406 (Manifold Regulator Readback Pressure),96.533771
9224 (Stack HPHT Pressure - Raw (PSI)),9224 (Stack HPHT Pressure - Raw (PSI)),96.505242
1414 (Lower Anuular Regulator Pilot Pressure),1414 (Lower Anuular Regulator Pilot Pressure),95.948934
9477 (LMRP HPHT Pressure - Corrected (PSI)),9477 (LMRP HPHT Pressure - Corrected (PSI)),95.556665
9277 (LMRP HPHT Pressure - Corrected (PSI)),9277 (LMRP HPHT Pressure - Corrected (PSI)),95.399758
9427 (LMRP HPHT Pressure - Raw (PSI)),9427 (LMRP HPHT Pressure - Raw (PSI)),95.371229
9227 (LMRP HPHT Pressure - Raw (PSI)),9227 (LMRP HPHT Pressure - Raw (PSI)),94.836317


In [7]:
# Substituir por "1" valores que tiveram modificacoes entre registros
df_filtered_full = df_filtered.fillna(0)
df_filtered_changes = df_filtered_full.diff()
df_filtered_changes[df_filtered_changes!=0] = 1
df_filtered_changes = df_filtered_changes.drop('2022-12-08T18:00:00.000Z',axis = 0)
df_filtered_changes

,0 (Pod Select),16 (Pod 1 Flow Meter - Total 1),17 (Pod 2 Flow Meter - Total 1),45 (Alarm Lamp),47 (Alarm Horn),250 (Test Pod SEM-A Serial Software Process - CCU Node # - Primary),251 (Test Pod SEM-A Serial Software Process - CCU Node # - Secondary),252 (Test Pod SEM-B Serial Software Process - CCU Node # - Primary),253 (Test Pod SEM-B Serial Software Process - CCU Node # - Secondary),1305 (SC Manifold Regulator Pressure - Decrease),...,9532 (SmV Ext. Function: Lower Inner Kill Valve - Close),9533 (SmV Ext. Function: Lower Inner Kill Valve - Open),9540 (SFS Ext. Function: Lower Outer Kill Valve - Close),9541 (SFS Ext. Function: Lower Outer Kill Valve - Open),9542 (SFS Ext. Function: Lower Inner Kill Valve - Close),9543 (SFS Ext. Function: Lower Inner Kill Valve - Open),9550 (External Pod Command State - Solenoid 1),9551 (External Pod Command State - Solenoid 2),9552 (External Pod Command State - Solenoid 3),9553 (External Pod Command State - Solenoid 4)
TIME (Date and Time),,,,,,,,,,,,,,,,,,,,,
2022-12-08T18:00:01.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:02.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:03.000Z,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:03.000Z,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-08T18:00:04.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-10T01:57:22.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-10T01:59:17.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-10T02:00:39.000Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
## Now with the sparse matrix, lets use the Association methodology
#!pip install mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

In [9]:
# Usando o modelo com o parametro support 0.01
frequent_itemsets = fpgrowth(df_filtered_changes, min_support=0.01, use_colnames=True)
frequent_itemsets

C:\Users\gustavo.araujo\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.251962,(2922 (60VDC Voltage Readout))
1,0.152019,(2904 (Manifold Annular Regulator Pilot Pressu...
2,0.053717,(1412 (Lower Annular Regulator Readback Pressu...
3,0.041518,(1418 (Wellhead Connector Regulator Pilot Pres...
4,0.139749,(2907 (Upper Annular Regulator Pilot Pressure))
...,...,...
191,0.011628,"(9215 (Calc Flexjoint Differential Bearing), 9..."
192,0.026751,"(2938 (Processor Temperature - Local), 2922 (6..."
193,0.010772,"(2938 (Processor Temperature - Local), 2919 (W..."
194,0.011628,(2907 (Upper Annular Regulator Pilot Pressure)...


In [10]:
#Adicianando a coluna de quantidade de itens relacionados
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.251962,(2922 (60VDC Voltage Readout)),1
1,0.152019,(2904 (Manifold Annular Regulator Pilot Pressu...,1
2,0.053717,(1412 (Lower Annular Regulator Readback Pressu...,1
3,0.041518,(1418 (Wellhead Connector Regulator Pilot Pres...,1
4,0.139749,(2907 (Upper Annular Regulator Pilot Pressure)),1
...,...,...,...
191,0.011628,"(9215 (Calc Flexjoint Differential Bearing), 9...",2
192,0.026751,"(2938 (Processor Temperature - Local), 2922 (6...",2
193,0.010772,"(2938 (Processor Temperature - Local), 2919 (W...",2
194,0.011628,(2907 (Upper Annular Regulator Pilot Pressure)...,2


In [11]:
#Filtrando se a associacao for menor que dois itens
df_assoc = frequent_itemsets[(frequent_itemsets['length'] >= 2)].sort_values(by='length', ascending=False)
df_assoc

,support,itemsets,length
168,0.012627,"(9224 (Stack HPHT Pressure - Raw (PSI)), 9227 ...",4
75,0.028820,(2907 (Upper Annular Regulator Pilot Pressure)...,3
114,0.011414,"(2922 (60VDC Voltage Readout), 1422 (60VDC Vol...",3
109,0.011129,"(9215 (Calc Flexjoint Differential Bearing), 9...",3
161,0.013340,"(9224 (Stack HPHT Pressure - Raw (PSI)), 9227 ...",3
...,...,...,...
105,0.012983,"(1406 (Manifold Regulator Readback Pressure), ...",2
104,0.014339,"(1406 (Manifold Regulator Readback Pressure), ...",2
103,0.016479,"(1406 (Manifold Regulator Readback Pressure), ...",2
102,0.011985,(1419 (Wellhead Connector Regulator Readback P...,2


In [12]:
#Imprimindo o resultado em um excel
df_assoc.to_csv(r'C:\Users\gustavo.araujo\OneDrive - SafeKick\Desktop\pessoal\Pos\ICA\Projeto final\BOP Monitoring\associate_FP_0.01.csv')

In [14]:
# Usando o modelo , mas agora com o parametro support 0.1
frequent_itemsets = fpgrowth(df_filtered_changes, min_support=0.1, use_colnames=True)
frequent_itemsets

C:\Users\gustavo.araujo\Anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.251962,(2922 (60VDC Voltage Readout))
1,0.152019,(2904 (Manifold Annular Regulator Pilot Pressu...
2,0.139749,(2907 (Upper Annular Regulator Pilot Pressure))
3,0.250250,(1422 (60VDC Voltage Readout))
4,0.132330,(1407 (Upper Annular Regulator Pilot Pressure))
5,0.143102,(2919 (Wellhead Connector Regulator Readback P...
6,0.142816,(1404 (Manifold Annular Regulator Pilot Pressu...
7,0.121344,(1419 (Wellhead Connector Regulator Readback P...
8,0.101512,(9227 (LMRP HPHT Pressure - Raw (PSI)))
9,0.103866,(1408 (Pod Pressure))


In [15]:
#Adicianando a coluna de quantidade de itens relacionados
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.251962,(2922 (60VDC Voltage Readout)),1
1,0.152019,(2904 (Manifold Annular Regulator Pilot Pressu...,1
2,0.139749,(2907 (Upper Annular Regulator Pilot Pressure)),1
3,0.250250,(1422 (60VDC Voltage Readout)),1
4,0.132330,(1407 (Upper Annular Regulator Pilot Pressure)),1
5,0.143102,(2919 (Wellhead Connector Regulator Readback P...,1
6,0.142816,(1404 (Manifold Annular Regulator Pilot Pressu...,1
7,0.121344,(1419 (Wellhead Connector Regulator Readback P...,1
8,0.101512,(9227 (LMRP HPHT Pressure - Raw (PSI))),1
9,0.103866,(1408 (Pod Pressure)),1


In [ ]:
#Percebe-se que o suporte deve ser muito baixo para obter-se alguma associacao. Comprovando que os valores em geral sao pouco frequentes na base de dados.